<a href="https://colab.research.google.com/github/ekacs/analytics_pelayanan_inaportnet_22/blob/main/analytics_inaportnet_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload File Scrape [format csv]

In [3]:
#install library wajib
!pip install pytz
import numpy as np
import pytz
import pandas as pd
import io
import datetime
from datetime import datetime, timedelta
from dateutil.tz import tzutc, tzlocal

#syntax upload file dari luar
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['rkbm tahun 22 samarinda.csv'])) #diisi nama file yg diupload

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#hitung selisih waktu respon dengan waktu permohonan

In [ ]:
#convert start_date to DateTime format, multiple columns
df['waktu permohonan'] = pd.to_datetime(df['waktu permohonan'])
df['waktu respon'] = pd.to_datetime(df['waktu respon'])
df['jam_permohonan_UTC'] = df['waktu permohonan']+pd.DateOffset(hours=8)
df['selisih_waktu'] = df['waktu respon']-df['jam_permohonan_UTC']

# Upload File dari operator [format xlsx]

In [ ]:
from google.colab import files
uploaded = files.upload()
df1 = pd.read_excel(io.BytesIO(uploaded['RKBM.IDSRI.2022-01-01.xlsx']))

In [ ]:
from google.colab import files
uploaded = files.upload()
df2 = pd.read_excel(io.BytesIO(uploaded['RKBM.IDSRI.2022-02-01.xlsx']))

In [ ]:
from google.colab import files
uploaded = files.upload()
df3 = pd.read_excel(io.BytesIO(uploaded['RKBM.IDSRI.2022-03-01.xlsx']))

In [ ]:
from google.colab import files
uploaded = files.upload()
df4 = pd.read_excel(io.BytesIO(uploaded['RKBM.IDSRI.2022-04-01.xlsx']))

In [ ]:
from google.colab import files
uploaded = files.upload()
df5 = pd.read_excel(io.BytesIO(uploaded['RKBM.IDSRI.2022-05-01.xlsx']))

In [ ]:
from google.colab import files
uploaded = files.upload()
df6 = pd.read_excel(io.BytesIO(uploaded['RKBM.IDSRI.2022-06-01.xlsx']))

In [ ]:
from google.colab import files
uploaded = files.upload()
df7 = pd.read_excel(io.BytesIO(uploaded['RKBM.IDSRI.2022-07-01.xlsx']))

In [ ]:
from google.colab import files
uploaded = files.upload()
df8 = pd.read_excel(io.BytesIO(uploaded['RKBM.IDSRI.2022-08-01.xlsx']))

In [ ]:
df_operator = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])
datamerge = pd.merge(df, df_operator, left_on=['nomor produk'], right_on=['NOMOR RKBM'], how='inner')

# mengambil data tanpa bug

In [ ]:
sort = datamerge.sort_values('selisih waktu', ascending = False)
datamerge_tanpabug = datamerge[datamerge['selisih waktu']>timedelta(seconds=0)]
datamerge_tanpabug.sort_values('selisih waktu', ascending = True).head()

#make criteria in time difference

In [ ]:
def sel(datamerge):
    if datamerge['selisih waktu'] < timedelta(seconds=31):
        return "Kurang dari 30 detik"
    elif datamerge['selisih waktu'] < timedelta(minutes=30):
        return "Antara 30 detik - 30 menit"
    elif datamerge['selisih waktu'] < timedelta(hours=1):
        return "Antara 30 - 60 menit"
    elif datamerge['selisih waktu'] < timedelta(hours=6):
        return "Antara 1 - 6 jam"
    elif datamerge['selisih waktu'] < timedelta(hours=12):
        return "Antara 6 - 12 jam"
    elif datamerge['selisih waktu'] < timedelta(days=1):
        return "Antara 12 - 24 jam"
    else:
        return "Lebih dari 24 jam"

datamerge_tanpabug['pembagian selisih waktu'] = datamerge_tanpabug.apply(sel, axis=1)
sort_data = datamerge_tanpabug.sort_values("selisih waktu", axis = 0, ascending = True)

#grouping pembagian selisih waktu

In [ ]:
waktu_pelayanan = datamerge_tanpabug.groupby('pembagian selisih waktu')['pembagian selisih waktu'].count().reset_index(name='jumlah')
waktu_pelayanan['persentase'] = waktu_pelayanan['jumlah']/len(datamerge_tanpabug)*100
waktu_pelayanan.head(7)

#indentasi penomoran waktu pelayanan

In [ ]:
def nomor_waktu(waktu_pelayanan):
    if waktu_pelayanan['pembagian selisih waktu'] == 'Kurang dari 30 detik':
        return 1
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 30 detik - 30 menit':
        return 2
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 30 - 60 menit':
        return 3
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 1 - 6 jam':
        return 4
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 6 - 12 jam':
        return 5
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 12 - 24 jam':
        return 6
    else:
        return 7

#grouping waktu pelayanan

In [ ]:
waktu_pelayanan['Nomor'] = waktu_pelayanan.apply(nomor_waktu, axis=1)
waktu_pelayanan = waktu_pelayanan.sort_values("Nomor", axis = 0, ascending = True)

#membuat table hari, bulan, waktu permohonan
#divide waktu permohonan into day, month, time

In [ ]:
month_mapping={
    1: 'January', 
    2: 'February', 
    3: 'March', 
    4: 'April', 
    5: 'May',
    6: 'June', 
    7: 'July',
    8: 'August'
} 
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 

datamerge_tanpabug['bulan permohonan'] = datamerge_tanpabug['waktu permohonan LT'].dt.month.map(month_mapping)
datamerge_tanpabug['hari permohonan'] = datamerge_tanpabug['waktu permohonan LT'].dt.dayofweek.map(dw_mapping)


#grouping pembagian bulan permohonan


In [ ]:
bulan_permohonan = datamerge_tanpabug.groupby('bulan permohonan')['bulan permohonan'].count().reset_index(name='jumlah')
bulan_permohonan['persentase'] = bulan_permohonan['jumlah']/len(datamerge)*100

def nomor(bulan_permohonan):
    if bulan_permohonan['bulan permohonan'] == 'January':
        return 1
    elif bulan_permohonan['bulan permohonan'] == 'February':
        return 2
    elif bulan_permohonan['bulan permohonan'] == 'March':
        return 3
    elif bulan_permohonan['bulan permohonan'] == 'April':
        return 4
    elif bulan_permohonan['bulan permohonan'] == 'May':
        return 5
    elif bulan_permohonan['bulan permohonan'] == 'June':
        return 6
    elif bulan_permohonan['bulan permohonan'] == 'July':
        return 7
    else:
        return 8

bulan_permohonan['nomor'] = bulan_permohonan.apply(nomor, axis=1)
bulan_permohonan = bulan_permohonan.sort_values("nomor", axis = 0, ascending = True)
bulan_permohonan


#grouping pembagian bulan permohonan

In [ ]:
def nomor_hari(hari_permohonan):
    if hari_permohonan['hari permohonan'] == 'Monday':
        return 1
    elif hari_permohonan['hari permohonan'] == 'Tuesday':
        return 2
    elif hari_permohonan['hari permohonan'] == 'Wednesday':
        return 3
    elif hari_permohonan['hari permohonan'] == 'Thursday':
        return 4
    elif hari_permohonan['hari permohonan'] == 'Saturday':
        return 5
    elif hari_permohonan['hari permohonan'] == 'Sunday':
        return 6
    else:
        return 7
    
hari_permohonan = datamerge_tanpabug.groupby('hari permohonan')['hari permohonan'].count().reset_index(name='jumlah')
hari_permohonan['Nomor'] = hari_permohonan.apply(nomor_hari, axis=1)
hari_permohonan = hari_permohonan.sort_values("Nomor", axis = 0, ascending = True)
hari_permohonan


#pembagian kriteria berdasarkan waktu permohonan


In [ ]:
batas_waktu1 = pd.to_datetime('06:00:00', format='%H:%M:%S')
batas_waktu2 = pd.to_datetime('12:00:00', format='%H:%M:%S')
batas_waktu3 = pd.to_datetime('18:00:00', format='%H:%M:%S')

#kriteria jam permohonan

In [ ]:
def jam(datamerge_tanpabug):
    if datamerge_tanpabug['jam permohonan'] < batas_waktu1:
        return "Jam 00.00 s.d. 06.00"
    elif datamerge_tanpabug['jam permohonan'] < batas_waktu2:
        return "Jam 06.00 s.d. 12.00"
    elif datamerge_tanpabug['jam permohonan'] < batas_waktu3:
        return "Jam 12.00 s.d. 18.00"
    else:
        return "Jam 18.00 s.d. 24.00"

datamerge_tanpabug['jam permohonan'] = datamerge_tanpabug['waktu permohonan LT'].dt.time
datamerge_tanpabug['jam permohonan'] = pd.to_datetime(datamerge_tanpabug['jam permohonan'], format='%H:%M:%S')
datamerge_tanpabug['pembagian jam permohonan'] = datamerge_tanpabug.apply(jam, axis=1)

#grouping pembagian jam permohonan

In [ ]:
waktu_permohonan = datamerge_tanpabug.groupby('pembagian jam permohonan')['pembagian jam permohonan'].count().reset_index(name='jumlah')
waktu_permohonan['persentase'] = waktu_permohonan['jumlah']/len(datamerge_tanpabug)*100